### Import Packages

In [219]:
import pandas as pd
import olefile
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

### Load Raw Data from .xls File(s)

In [220]:
def xls_to_dataframe(path):
    '''
    This function is needed because the xlrd engine used by pandas' read_excel
    method (or if xlrd is used directly) will throw the following exception
    when attempting to read MyPlate detailed-level export .xls files:
    
    xlrd.compdoc.CompDocError: Workbook corruption: seen[2] == 4
    
    The following solution to get around this issue was inspired by:
    https://stackoverflow.com/a/60416081/3761560
    
    Before arriving at the below solution, attempting to read the MyPlate .xls with xlrd
    directly using the 'ignore_workbook_corruption=True' option, only enabled reading up to
    row 163 for several different files. However, the actual data appeared intact, i.e., I
    believe the reported "corruption" to be benign and is an artifact of how the .xls
    file was generated and / or a limitation of xlrd (depending up on your viewpoint) and
    is related to "Compound File Binary" compatibility.
    
    For further reference, see:
    https://web.archive.org/web/20190311101348/http://www.crimulus.com/2013/09/19/reading-compound-file-binary-format-files-generated-by-phpexcel-with-pythondjango-xlrd/
    '''
    with open(path,'rb') as file:
        ole = olefile.OleFileIO(file)
        if ole.exists('Workbook'):
            d = ole.openstream('Workbook')
            return pd.read_excel(d, engine='xlrd', header=None)

filename = 'MyPlate-Export-2023-01-01_detailed.xls'
df_raw = xls_to_dataframe(filename)
df_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,Date:,"April 19th, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Meals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Meal,Item Brand,Item Name,Total Servings,Total Calories,Total Calories From Fat,Total Calories Percent Fat,Total Calories Percent Carbs,Total Calories Percent Protein,Total Fat,...,Niacin,Manganese,Iodine,Chromium,Molybdenum,Folic Acid,Copper,Vitamin E,Vitamin K,Trans Fat
4,breakfast,NaN,"Muffin, Blueberry",1,359,43,0,0,0,4.82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Parse and Rearrange Raw Data(frame)

* Note: Initial project will just focus on meals, but later versions will / may incorporate water and fitness data

In [221]:
MEAL_NAMES = ["breakfast", "lunch", "dinner", "snacks"]

df_meals   = pd.DataFrame()

# Track current date. Also track the adjacent reference row, which contains
# the nutritional labels (e.g., 'Meal', 'Item Brand', 'Total Calories', ...
# 'Niacin', 'Chromium', 'Folic Acid', etc...) to be used as headers in the
# final table. Always tracking the latest / adjacent nutrition label reference
# row, rather than assuming all are the same, will guard against misplaced
# nutritional values if the daily sub-table alignment and / or set of nutritional
# labels should ever change.

curr_date = None
curr_meal_ref_row = None

# Check all rows because we have to
for i in range(len(df_raw)):
    
    # First column value as "label"
    label = df_raw.iloc[i, 0]
    
    # Set current date
    if label == "Date:":
        curr_date = df_raw.iloc[i, 1]
    
    # Extract meal info
    if label == "Meal":
        curr_meal_ref_row = df_raw.iloc[i]
        curr_meal_ref_row = ["Date"] + list(curr_meal_ref_row)
    
    if label in MEAL_NAMES:
        curr_meal_value_row = [curr_date] + list(df_raw.iloc[i])
        new_row_dict = dict(zip(curr_meal_ref_row, curr_meal_value_row))
        df_meals = df_meals.append(new_row_dict, ignore_index=True)


In [222]:
df_meals.head()

,Date,Meal,Item Brand,Item Name,Total Servings,Total Calories,Total Calories From Fat,Total Calories Percent Fat,Total Calories Percent Carbs,Total Calories Percent Protein,...,Niacin,Manganese,Iodine,Chromium,Molybdenum,Folic Acid,Copper,Vitamin E,Vitamin K,Trans Fat
0,"April 19th, 2023",breakfast,NaN,"Muffin, Blueberry",1.0,359.0,43.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"April 19th, 2023",breakfast,NaN,Brewed Coffee,1.0,2.0,0.0,NaN,NaN,NaN,...,0.45mg,0.05mg,NaN,NaN,NaN,0µg,0mg,0.02mg,0.24µg,0g
2,"April 19th, 2023",breakfast,NaN,brewed decaf coffee,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"April 19th, 2023",breakfast,NaN,Milk Whole,0.5,74.5,35.5,NaN,NaN,NaN,...,0.22mg,0.01mg,NaN,NaN,NaN,0µg,0.06mg,0.17mg,0.73µg,NaN
4,"April 19th, 2023",breakfast,Organic Valley,"Extra-Large Brown Eggs, Organic",2.0,140.0,80.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0g


In [223]:
df_meals.tail()

,Date,Meal,Item Brand,Item Name,Total Servings,Total Calories,Total Calories From Fat,Total Calories Percent Fat,Total Calories Percent Carbs,Total Calories Percent Protein,...,Niacin,Manganese,Iodine,Chromium,Molybdenum,Folic Acid,Copper,Vitamin E,Vitamin K,Trans Fat
1624,"January 1st, 2023",snacks,NaN,Black Decaf Tea,1.00,2.00,0.00,NaN,NaN,NaN,...,0mg,0.52mg,NaN,NaN,NaN,0µg,0.02mg,0mg,0µg,0g
1625,"January 1st, 2023",snacks,NaN,Milk Whole,0.33,49.17,23.43,NaN,NaN,NaN,...,0.22mg,0.01mg,NaN,NaN,NaN,0µg,0.06mg,0.17mg,0.73µg,NaN
1626,"January 1st, 2023",snacks,Tillamook,"Cheese, Sliced Sharp Cheddar",1.00,120.00,90.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0g
1627,"January 1st, 2023",snacks,Thomas',Pita Pockets 100% Whole Wheat,1.00,140.00,15.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1628,"January 1st, 2023",snacks,NaN,Honey,1.00,43.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
